In [1]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image

# Initialize the processor and model
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-large-handwritten")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-large-handwritten")

preprocessor_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.23G [00:00<?, ?B/s]

VisionEncoderDecoderModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-large-handwritten and are newly initialized: [

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
# Specify the path to your local image
image_path = r'/kaggle/input/qwertyuiop/r03-030-s00-00.png' # Replace with your actual image path

# Load the image from the local file system
try:
    image = Image.open(image_path).convert("RGB")
except FileNotFoundError:
    print(f"Error: The file at {image_path} was not found.")
    exit(1)

# Process the image and generate text
pixel_values = processor(image, return_tensors="pt").pixel_values
generated_ids = model.generate(pixel_values)

# Decode the generated text
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

print("Generated Text:")
print(generated_text)

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Generated Text:
The next thing that Serena discovered was that


In [4]:
!pip install sympy
!pip install language-tool-python
!pip install reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 11.7 MB/s eta 0:00:00


In [5]:
import re
import sympy as sp
import language_tool_python
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.units import inch

# Function to extract LaTeX math and convert it into text
def latex_to_text(latex_str):
    # Use sympy to parse LaTeX for math expressions
    try:
        expr = sp.sympify(latex_str)
        return str(expr)
    except (sp.SympifyError, ValueError):
        # If parsing fails, return the original LaTeX as text
        return latex_str

# Function to correct grammar using LanguageTool
def correct_grammar(text):
    tool = language_tool_python.LanguageTool('en-US')  # Set the language
    matches = tool.check(text)
    corrected_text = language_tool_python.utils.correct(text, matches)
    return corrected_text

# Function to convert text to a PDF file
def text_to_pdf(text, output_pdf):
    c = canvas.Canvas(output_pdf, pagesize=letter)
    width, height = letter
    # Splitting text into multiple lines for PDF (line wrap)
    lines = text.split('\n')
    y = height - 40  # Top margin

    for line in lines:
        if y < 40:  # Bottom margin, start a new page
            c.showPage()
            y = height - 40

        # Add LaTeX rendering logic
        if '$' in line or '\\[' in line:  # Check if line contains LaTeX math
            # Render LaTeX here using a method of your choice
            c.drawString(40, y, line)  # This won't render math; requires more work
        else:
            c.drawString(40, y, line)
        y -= 15  # Line spacing

    c.save()

# Main function
def process_latex_input(latex_text, output_pdf):
    # Find all LaTeX math enclosed in $...$ or \[...\]
    latex_math = re.findall(r'\$[^\$]*\$|\\\[.*?\\\]', latex_text)

    # Replace LaTeX math with plain text equivalents
    for math_expr in latex_math:
        plain_text = latex_to_text(math_expr.strip('$').strip('\\[').strip('\\]'))
        latex_text = latex_text.replace(math_expr, plain_text)

    # Correct the grammar of the text
    corrected_text = correct_grammar(latex_text)

    # Convert the corrected text to a PDF
    text_to_pdf(corrected_text, output_pdf)
    print(f"PDF generated at: {output_pdf}")

# Example usage
# process_latex_input("Here is some math: $E=mc^2$ and here is text.", "output.pdf")


In [12]:

# # Example usage
# latex_input = """This is an example. The equation is $x^2 + y^2 = z^2$.
# We also have the integral $ \int_0^1 x^2 dx $."""
output_pdf_path = "outputtt.pdf"

# Process the LaTeX input and generate the PDF
process_latex_input(generated_text, output_pdf_path)


PDF generated at: outputtt.pdf
